## System dynamics
CartPole environment consists of a cart on a trail and a pole hinge fixed on it. 
<img src="./img/Environment.png" width="300">
### States
$$X = \left[\array{x \\ \dot{x} \\ \theta \\ \dot{\theta} }\right]$$
### Derivative equations based on Newton and Euler's equations
$$(M+m)\ddot{x}= F_u + ml (\dot{\theta})^2sin\theta-ml\ddot{\theta}cos\theta$$  
$$\frac{3}{4} ml^2\ddot{\theta}=mglsin\theta-mlcos\theta \ddot{x}$$
- Parameters: 
l is distance from joint point to mass center of pole
- Combination in discrete time space
$$\delta t=\tau$$
$$\gamma = \frac{(F_u+ml{\dot{\theta}}^2sin\theta)}{M+m}$$   
$$\ddot{\theta}=\frac{gsin\theta-\gamma cos\theta}{\frac{4}{3}(M+m)l-mlcos^2\theta}$$
$$\ddot{x}=\gamma-\frac{ml\ddot{\theta}cos\theta}{M+m}$$
- In very short discrete time interval
$$x = x + \tau\dot{x}$$  
$$\dot{x}=\dot{x}+\tau\ddot{x}$$  
$$\theta = \theta + \tau\dot{\theta}$$  
$$\dot{\theta}=\dot{\theta}+\tau\ddot{\theta}$$
### Linearized state space function     
- Assumption  
$$sin\theta\approx\theta$$
$$cos\theta\approx 1$$
$$(\dot{\theta})^2 \approx 0$$  
- Take assumption into derivative equations
$$\beta = \frac{4}{3}-\frac{ml}{M+m}$$  
$$\ddot{x}=-\frac{mlg\theta}{(M+m)\beta}\theta-\frac{F_u(\beta(M+m)+ml)}{(M+m)^2l\beta}$$  
$$\ddot{\theta}=\frac{g}{l\beta}\theta-\frac{F_u}{(M+m)l\beta}$$  
- The linearized discrete equation of system following previous procedure
    - states
$$X=\left[\matrix{x \\ \theta \\ \dot{x} \\ \dot{\theta}}\right]$$  
    - state equations
$$x_{k+1}=x_k+\dot{x_k}\tau$$  
$$\theta_{k+1}=\theta_k+\dot{\theta_k}\tau$$  
$$\dot{x}_{k+1} = \dot{x}_k+\tau (-\frac{mlg\theta_k}{(M+m)\beta}\theta_k-\frac{F_{u_k}(\beta(M+m)+ml)}{(M+m)^2l\beta})$$  
$$\dot{\theta}_{k+1} = \dot{\theta}_k+\tau(\frac{g}{l\beta}\theta_k-\frac{F_{u_k}}{(M+m)l\beta})$$  
    - discrete state space
$$\left[\matrix{x_{k+1} \\ \theta_{k+1} \\ \dot{x}_{k+1} \\ \dot{\theta}_{k+1}}\right] = 
\left[\matrix{1 & \tau & 0 & 0 \\  
              0 & 1 & -\frac{\tau mlg}{(M+m)\beta} & 0 \\  
              0 & 0 & 1 & \tau \\  
              0 & 0 & \frac{g\tau}{l\beta_k}& 1}\right]\left[\matrix{x_{k} \\ \theta_{k} \\ \dot{x}_{k} \\ \dot{\theta}_{k}}\right] + \left[\matrix{0 \\ \frac{(\beta(M+m)+ml)}{(M+m)^2l\beta} \\ 0 \\ \frac{-1}{(M+m)l\beta} }\right][F_{u_k}]$$          